In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import importlib
import mr_rrn, data_aux
from tensorflow.contrib import learn

In [ ]:
# Settings test
examples=50000
num_steps = 20
num_seq = 4
n_hidden_enc = 4
n_hidden_con = 5
n_hidden_dec = 8
batch_size = 100
learning_rate = 0.05
embedding_shape = [4, 3] # [voc_size, emb_dim]

In [ ]:
# Settings real deal
num_steps = 20
num_seq = 6
n_hidden_enc = 300
n_hidden_con = 300
n_hidden_dec = 300
batch_size = 40
learning_rate = 0.0002

In [ ]:
importlib.reload(mr_rrn)
tf.reset_default_graph()
coarse_kwargs = {'embedding_shape' : [2,3],
                 'num_seq' : num_seq,
                 'num_steps' : 20,
                 'batch_size' : batch_size,
                 'n_hidden_encoder' : 3,
                 'n_hidden_context' : 4,
                 'n_hidden_decoder' : 5}
nl_kwargs = {'embedding_shape' : [3,4],
               'num_seq' : num_seq,
               'num_steps' : 12,
               'batch_size' : batch_size,
               'n_hidden_encoder' : 6,
               'n_hidden_context' : 7, 
               'n_hidden_decoder' : 8}
graph_nodes = mr_rrn.build_graph(coarse_kwargs, 2, nl_kwargs, learning_rate, gradient_clipping = 1.)

In [ ]:
# Load data
importlib.reload(mr_rrn)

# Simple test
data_sequences, data_len = data_aux.generate_test_data_sequence(examples=examples, num_seq=num_seq, num_steps=num_steps)
W_embedding = np.eye(4)
# The real deal
#data_sequences, data_len, vocab_processor = data_aux.generate_data_ubuntu(max_utterances = num_seq, max_tokens = num_steps)
#W_embedding = data_aux.random_embedding(vocab_processor)

embedding_shape = W_embedding.shape
batches = data_aux.batch_iter([data_sequences, data_len],batch_size=batch_size, num_epochs=5)

# Build graph
timer = datetime.now()
tf.reset_default_graph()
hred_kwargs = {'embedding_shape' : embedding_shape,
               'num_seq' : num_seq,
               'num_steps' : num_steps,
               'batch_size' : batch_size,
               'n_hidden_encoder' : n_hidden_enc,
               'n_hidden_context' : n_hidden_con, 
               'n_hidden_decoder' : n_hidden_dec}
graph_nodes = mr_rrn.build_graph(hred_kwargs, learning_rate, gradient_clipping = 1.)
timer = datetime.now() - timer
print('Time build graph: ', timer)

# Train
acc_loss = 0
with tf.Session() as sess:
    timer = datetime.now()
    sess.run(tf.initialize_all_variables(), feed_dict={graph_nodes['vocab_input'] : W_embedding})
    timer = datetime.now() - timer
    print('Time init graph: ', timer)
    timer = datetime.now()
    acc_loss = 0
    for i, batch in enumerate(batches):
        seq_batch, len_batch = batch
        feed_dict = {graph_nodes['sequence_input'] : seq_batch,
                     graph_nodes['length_input'] : len_batch
                     }
        loss, _ = sess.run([graph_nodes['total_loss'], graph_nodes['train_step']], feed_dict=feed_dict)
        acc_loss += loss
        if i % 25 == 0 and i > 0:
            print(acc_loss/25)
            acc_loss = 0
timer = datetime.now() - timer
print('Time run graph: ', timer)